## Fetching Data from an Itunes XML DUMP File and storing it an sqlite database

In this Project we take an XML version of an Itunes Playlist dump and fetch important data points to be stored in an SQL database to retrieve and analysze further. An illustration of how we can make raw data into actionable data points that can be used to draw further insights on it. 

In [26]:
#Importing the library
import xml.etree.ElementTree as ET
import sqlite3
#Importing the connector and cursor
conn = sqlite3.connect('trackdb2.sqlite')
cur = conn.cursor()

In [27]:
#Creating the required schema of the database
cur.executescript('''
drop table if exists Artist;
drop table if exists Album;
drop table if exists Genre; 
drop table if exists Track;

Create Table Artist (
id integer not null primary key  autoincrement unique, 
name text unique
);

Create Table Genre(
id integer not null primary key autoincrement unique,  
name text unique
); 


Create table Album(
id integer not null primary key autoincrement  unique, 
artist_id integer, 
title text unique
);

Create Table Track(
id integer not null primary key autoincrement  unique, 
title text unique,
album_id integer, 
genre_id integer, 
len interger, rating integer, count integer
)

''')

In [28]:
#Defining a function to look up to the required datapoints in the xml file
fname = input("Enter the name of the file:")
if (len(fname)<1): fname='Library.xml'
    
def lookup(d, key):
    found = False
    for child in d: 
        if found: return child.text
        if child.tag =='key' and child.text == key:
            found = True
    return None

new_var = ET.parse(fname)
new = new_var.findall('dict/dict/dict')

print('Number of Records:', len(new))

#Fetching the required data points from the xml
for x in new: 
    if (lookup(x, 'Track ID') is None): continue 
        
    name = lookup(x, 'Name')
    artist = lookup(x, 'Artist')
    album = lookup(x, 'Album')
    genre = lookup(x, 'Genre')
    track = lookup(x, 'Track')
    length = lookup(x, 'Total Time')
    rating = lookup(x, 'Rating')
    count = lookup(x, 'Play Count')
    
    if name is None or artist is None or album is None or genre is None: 
        continue 
    
    print(name, artist, album, genre, length, rating, count)
    
    #Storing the above data points in the database schema
    cur.execute('''insert or ignore into Artist (name) values (?)''', ( artist,))
    cur.execute('Select id from Artist where name = ?',(artist,))
    artist_id = cur.fetchone()[0]
    
    cur.execute('insert or ignore into Genre (name) Values (?)',(genre,))
    cur.execute('select id from Genre where name = ?',(genre, ))
    genre_id = cur.fetchone()[0]
    
    cur.execute('insert or ignore into Album (artist_id, title) Values (?,?)', (artist_id, album))
    cur.execute('select id from Album where title = ? ', (album,))
    album_id = cur.fetchone()[0]
    
    cur.execute('''insert or ignore into Track (title, album_id, genre_id, len, rating, count)
    Values (?,?,?,?,?,?) ''',(name, album_id, genre_id, length, rating, count))
    
    conn.commit()

Enter the name of the file:
Number of Records: 404
Another One Bites The Dust Queen Greatest Hits Rock 217103 100 55
Asche Zu Asche Rammstein Herzeleid Industrial 231810 100 79
Beauty School Dropout Various Grease Soundtrack 239960 100 48
Black Dog Led Zeppelin IV Rock 296620 100 109
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] Rock 87118 100 33
Circles Bryan Lee Blues Is Funk 355369 60 54
Comfortably Numb Pink Floyd The Wall [Disc 2] Rock 384130 100 36
Crazy Little Thing Called Love Queen Greatest Hits Rock 163631 100 38
Electric Funeral Black Sabbath Paranoid Metal 293015 100 44
Fat Bottomed Girls Queen Greatest Hits Rock 257515 100 38
For Those About To Rock (We Salute You) AC/DC Who Made Who Rock 353750 100 84
Four Sticks Led Zeppelin IV Rock 284421 100 84
Furious Angels Rob Dougan The Matrix Reloaded Soundtrack 330004 100 54
Gelle Bryan Lee Blues Is Blues/R&B 199836 60 45
Going To California Led Zeppelin IV Rock 215666 100 100
Grease Various Grease Soundtrack 205792 100 4

Pahrump-Big Water Cusco Natural Wonders Music Sampler 1999 New Age 191634 None 342
Seeker's Quest Cheryl Gunn Natural Wonders Music Sampler 1999 New Age 236434 None 337
Floating To Forever Dean Everson Natural Wonders Music Sampler 1999 New Age 351686 None 337
Open Road Jeff Bailey Relaxing Jazz Jazz 318563 None 10
Ruby Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 205008 None 13
The Angry Mob Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 288313 None 41
Heat Dies Down Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 237061 None 12
Highroyds Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 199575 None 10
Love's Not A Competition (But I'm Winning) Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 197799 None 11
Thank You Very Much Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 157753 None 11
I Can Do It Without You Kaiser Chiefs Yours Truly, Angry Mob Alternative & Punk 204199 None 11
My Kind Of Guy Kaiser Chiefs Yours Truly, Angry Mob Alt

Katie Hafner: The Origins of the Internet IEEE Computer Society Computing Conversations Podcast 652460 None 1
Larry Smarr: Building Mosaic IEEE Computer Society Computing Conversations Podcast 894693 None None
Len Kleinrock: The Theory of Packets IEEE Computer Society Computing Conversations Podcast 674742 None 1
Mitchell Baker: The Mozilla Foundation IEEE Computer Society Computing Conversations Podcast 994246 None None
Pooja Sankar: Building the Piazza Collaboration System IEEE Computer Society Computing Conversations Podcast 496404 None None
Van Jacobson: Content-Centric Networking IEEE Computer Society Computing Conversations Podcast 780251 None None
The Apache Software Foundation IEEE Computer Society Computing Conversations Podcast 542484 None None
A Brief History of Packets IEEE Computer Society Computing Conversations Podcast 1004643 None None
Discovering JavaScript Object Notation IEEE Computer Society Computing Conversations Podcast 650971 None None
Inventing PHP IEEE Compute

Song for Someone U2 Songs of Innocence Rock 226763 None None
The Miracle (Of Joey Ramone) U2 Songs of Innocence Rock 255382 None None
Iris (Hold Me Close) U2 Songs of Innocence Rock 319457 None None
Goodbye Daughters of the Revolution The Black Crowes Warpaint Rock 304147 None None
Walk Believer Walk The Black Crowes Warpaint Rock 280253 None None
Choice Hotels International Undercover Boss Undercover Boss, Season 2 Reality TV 2614047 None 1
Microchip Jason Farnham YouTube Audio Library Dance & Electronic 114336 None None
Anant Agarwal IEEE Computer Society Computing Conversations Podcast 494000 None None
